# Objective and Motivation

## Objective

## Overview of Wind Turbine Anatomy

## Wind Turbine Aerodynamics

# Code

### Pip Install, Init, and Start Pyspark

In [59]:
# from pyspark.sql import SparkSession
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# from pyspark.sql import functions as F
# from pyspark.sql import types as T
# from pyspark.sql.functions import broadcast
# from pyspark.sql.functions import col
# from pyspark.sql.functions import *
import seaborn as sns

%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

In [60]:
# from ydata_profiling import ProfileReport

In [61]:
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# sc = spark.sparkContext

In [62]:
# Get the number of cores
# num_cores = sc.defaultParallelism

# Get the number of executors
# num_executors = sc.getConf().get("spark.executor.instances")

# print(num_cores)
# print(num_executors)

In [63]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Import Data 

In [64]:
print(os.getcwd())

/home/mathewlee/Developer/wind-energy-forecasting/notebooks


In [65]:
df = pd.read_csv("../src/main/resources/data/Turbine_Data.csv")

In [66]:
# df = (spark.read
#       .option("header", "true")
#       .option("inferSchema", "true")
#       .csv("../data/Turbine_Data.csv"))

In [67]:
# df.printSchema()

In [68]:
# df_renamed = df.withColumnRenamed("_c0", "datetime_stamp")


In [69]:
df_renamed = df.rename({"Unnamed: 0": "datetime_stamp"}, axis=1)

In [70]:
df_renamed.columns

Index(['datetime_stamp', 'ActivePower', 'AmbientTemperatue',
       'BearingShaftTemperature', 'Blade1PitchAngle', 'Blade2PitchAngle',
       'Blade3PitchAngle', 'ControlBoxTemperature',
       'GearboxBearingTemperature', 'GearboxOilTemperature', 'GeneratorRPM',
       'GeneratorWinding1Temperature', 'GeneratorWinding2Temperature',
       'HubTemperature', 'MainBoxTemperature', 'NacellePosition',
       'ReactivePower', 'RotorRPM', 'TurbineStatus', 'WTG', 'WindDirection',
       'WindSpeed'],
      dtype='object')

In [71]:
df_renamed['datetime_stamp'] = pd.to_datetime(df_renamed['datetime_stamp'])

In [72]:
df_renamed.dtypes

datetime_stamp                  datetime64[ns, UTC]
ActivePower                                 float64
AmbientTemperatue                           float64
BearingShaftTemperature                     float64
Blade1PitchAngle                            float64
Blade2PitchAngle                            float64
Blade3PitchAngle                            float64
ControlBoxTemperature                       float64
GearboxBearingTemperature                   float64
GearboxOilTemperature                       float64
GeneratorRPM                                float64
GeneratorWinding1Temperature                float64
GeneratorWinding2Temperature                float64
HubTemperature                              float64
MainBoxTemperature                          float64
NacellePosition                             float64
ReactivePower                               float64
RotorRPM                                    float64
TurbineStatus                               float64
WTG         

In [73]:
pdf = df_renamed.copy()

In [74]:
# df_casted = df_renamed.withColumn("datetime_stamp", col("datetime_stamp").cast("timestamp")).drop(col("_c0"))
# df_casted.printSchema()

In [75]:
# df_filtered = df_casted.where(
#     to_date(col("datetime_stamp")) =="2018-01-01"
#     )

In [76]:
# pdf = df_casted.toPandas()

In [77]:
# profile = ProfileReport(pdf, title="YData Profiling Report")
# profile.to_file('profiling_report.html');

### Drop Empty or Single Value Cols

In [78]:
pdf_drop_bad_cols = pdf.drop(columns=['ControlBoxTemperature', 'TurbineStatus', 'WTG'], axis=1)

In [79]:
type(pdf_drop_bad_cols)

pandas.core.frame.DataFrame

In [80]:
corr_matrix = pdf_drop_bad_cols.corr()
corr_matrix


,datetime_stamp,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,WindDirection,WindSpeed
datetime_stamp,1.000000,-0.029146,-0.107832,-0.088521,0.032461,0.030255,0.030255,-0.111076,-0.197239,-0.091271,-0.145035,-0.145350,-0.137042,-0.074034,-0.062090,0.212018,-0.094242,-0.062090,-0.019445
ActivePower,-0.029146,1.000000,-0.065637,0.655400,-0.368997,-0.367978,-0.367978,0.818850,0.821921,0.849604,0.931390,0.932521,0.347363,0.101627,0.028108,0.719092,0.849069,0.028108,0.940390
AmbientTemperatue,-0.107832,-0.065637,1.000000,0.246505,0.085461,0.090904,0.090904,0.017216,0.162511,-0.130123,0.077859,0.078736,0.589729,0.836539,-0.037498,-0.020313,-0.129290,-0.037498,-0.095308
BearingShaftTemperature,-0.088521,0.655400,0.246505,1.000000,-0.475557,-0.466856,-0.466856,0.883344,0.772887,0.640947,0.765115,0.763926,0.809116,0.542104,0.183929,0.594546,0.640584,0.183929,0.596201
Blade1PitchAngle,0.032461,-0.368997,0.085461,-0.475557,1.000000,0.997774,0.997774,-0.599178,-0.557949,-0.757056,-0.376102,-0.372369,-0.198583,0.194911,-0.058300,-0.381478,-0.757147,-0.058300,-0.403092
Blade2PitchAngle,0.030255,-0.367978,0.090904,-0.466856,0.997774,1.000000,1.000000,-0.592103,-0.549792,-0.754325,-0.371057,-0.367451,-0.190296,0.190562,-0.057624,-0.379541,-0.754539,-0.057624,-0.402786
Blade3PitchAngle,0.030255,-0.367978,0.090904,-0.466856,0.997774,1.000000,1.000000,-0.592103,-0.549792,-0.754325,-0.371057,-0.367451,-0.190296,0.190562,-0.057624,-0.379541,-0.754539,-0.057624,-0.402786
GearboxBearingTemperature,-0.111076,0.818850,0.017216,0.883344,-0.599178,-0.592103,-0.592103,1.000000,0.906020,0.850209,0.853024,0.851923,0.596065,0.307694,0.214486,0.751875,0.850795,0.214486,0.800433
GearboxOilTemperature,-0.197239,0.821921,0.162511,0.772887,-0.557949,-0.549792,-0.549792,0.906020,1.000000,0.806972,0.899693,0.898335,0.506648,0.190350,0.277619,0.770088,0.807776,0.277619,0.793140
GeneratorRPM,-0.091271,0.849604,-0.130123,0.640947,-0.757056,-0.754325,-0.754325,0.850209,0.806972,1.000000,0.796434,0.795223,0.303804,-0.052185,0.176964,0.768065,0.999723,0.176964,0.854787


In [81]:
corr_matrix.abs().unstack()["NacellePosition"]

datetime_stamp                  0.062090
ActivePower                     0.028108
AmbientTemperatue               0.037498
BearingShaftTemperature         0.183929
Blade1PitchAngle                0.058300
Blade2PitchAngle                0.057624
Blade3PitchAngle                0.057624
GearboxBearingTemperature       0.214486
GearboxOilTemperature           0.277619
GeneratorRPM                    0.176964
GeneratorWinding1Temperature    0.298452
GeneratorWinding2Temperature    0.298708
HubTemperature                  0.128150
MainBoxTemperature              0.005701
NacellePosition                 1.000000
ReactivePower                   0.294038
RotorRPM                        0.176511
WindDirection                   1.000000
WindSpeed                       0.034355
dtype: float64

In [82]:
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates().head(20)

WindSpeed                     WindSpeed                       1.000000
GeneratorWinding2Temperature  GeneratorWinding1Temperature    0.999959
RotorRPM                      GeneratorRPM                    0.999723
Blade1PitchAngle              Blade2PitchAngle                0.997774
WindSpeed                     ActivePower                     0.940390
ActivePower                   GeneratorWinding2Temperature    0.932521
GeneratorWinding1Temperature  ActivePower                     0.931390
GearboxOilTemperature         GearboxBearingTemperature       0.906020
GeneratorWinding1Temperature  GearboxOilTemperature           0.899693
GearboxOilTemperature         GeneratorWinding2Temperature    0.898335
WindSpeed                     GeneratorWinding2Temperature    0.894834
GeneratorWinding1Temperature  WindSpeed                       0.894777
BearingShaftTemperature       GearboxBearingTemperature       0.883344
RotorRPM                      WindSpeed                       0.855584
Genera

In [86]:
### Drop highly correlated features
### pdf[["WindSpeed","ActivePower"]]
### pdf[["RotorRPM", "GeneratorRPM"]]
### pdf[["AmbientTemperatue", "MainBoxTemperature"]]
### pdf[["Blade1PitchAngle","Blade2PitchAngle", "Blade3PitchAngle"]]
### pdf[["NacellePosition","WindDirection"]]


df_drop_corr_cols = pdf.drop(["NacellePosition","RotorRPM", "AmbientTemperatue", "Blade2PitchAngle", "Blade3PitchAngle"], axis=1)


In [88]:
df_drop_corr_cols

,datetime_stamp,ActivePower,BearingShaftTemperature,Blade1PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,ReactivePower,TurbineStatus,WTG,WindDirection,WindSpeed
0,2017-12-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
1,2017-12-31 00:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
2,2017-12-31 00:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
3,2017-12-31 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
4,2017-12-31 00:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118219,2020-03-30 23:10:00+00:00,70.044465,45.711129,1.515669,0.0,59.821165,55.193793,1029.870744,59.060367,58.148777,39.008931,36.476562,13.775785,2.0,G01,178.0,3.533445
118220,2020-03-30 23:20:00+00:00,40.833474,45.598573,1.702809,0.0,59.142038,54.798545,1030.160478,58.452003,57.550367,39.006759,36.328125,8.088928,2.0,G01,178.0,3.261231
118221,2020-03-30 23:30:00+00:00,20.777790,45.462045,1.706214,0.0,58.439439,54.380456,1030.137822,58.034071,57.099335,39.003815,36.131944,4.355978,2.0,G01,178.0,3.331839
118222,2020-03-30 23:40:00+00:00,62.091039,45.343827,1.575352,0.0,58.205413,54.079014,1030.178178,57.795387,56.847239,39.003815,36.007805,12.018077,2.0,G01,190.0,3.284468
